In [ ]:
"""
Este script utiliza Python para automatizar la búsqueda y análisis de productos en plataformas de compraventa online.
Se conecta con una API de IA (GPT-4 de OpenAI) para extraer insights relevantes sobre los datos obtenidos.

⚠️ Importante:
- La búsqueda está limitada para cumplir con buenas prácticas y respetar los términos de uso de las plataformas.
- El objetivo del código es educativo y para el análisis de datos, fomentando el uso responsable de la automatización.

Funciones principales:
1. Extraer datos de productos según una consulta específica.
2. Analizar y filtrar los datos usando inteligencia artificial.
3. Generar informes claros sobre precios y tendencias.

Nota: Este script es una demostración con fines educativos. Antes de usar técnicas de scraping en plataformas reales, verifica y respeta sus términos y condiciones.
"""

# IN:linkedin.com/in/sanchezalonsodavid17

import openai
from openai import OpenAI
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import unicodedata
import urllib.parse
import time
import re
import json

class OpenAIHandler:
    def __init__(self, api_key):
        """
        Inicializa el manejador de OpenAI.
        Args:
            api_key (str): Clave API de OpenAI.
        """
        self.client = OpenAI(api_key=api_key)

    def filtrar_productos_por_tema(self, productos, query):
        filtrados = []
        for producto in productos:
            prompt = f"""
            Eres un asistente que analiza productos de Marketplace. Dado el siguiente producto:
            Título: {producto['titulo']}
            ¿Este producto está relacionado con la búsqueda '{query}'? Responde 'Sí' o 'No' únicamente.
            """
            try:
                respuesta = self.client.chat.completions.create(
                    model="gpt-4",
                    messages=[
                        {"role": "system", "content": "Eres un asistente que filtra productos por relevancia."},
                        {"role": "user", "content": prompt}
                    ],
                    max_tokens=5,
                    temperature=0.0
                )
                # Acceso al contenido del mensaje de forma correcta
                contenido = respuesta.choices[0].message.content.strip().lower()
                print(f"Respuesta de OpenAI para '{producto['titulo']}': {contenido}")  # Depuración
                if contenido in ["sí", "si"]:
                    filtrados.append(producto)
            except Exception as e:
                print(f"Error al filtrar producto: {producto['titulo']} - {e}")
        return filtrados



    def generar_insights(self, productos, query):
        prompt = f"""
        Eres un asistente que analiza oportunidades de compra y reventa en Marketplace.
        La búsqueda es: '{query}'.
        Los productos encontrados son:
        {json.dumps(productos, indent=2)}

        Por favor, identifica las mejores oportunidades de compra y reventa basadas en los datos anteriores. Tu respuesta debe ser breve y clara, destacando únicamente las conclusiones clave en un máximo de 5 líneas.
        """
        try:
            respuesta = self.client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "Eres un asistente experto en análisis de oportunidades de mercado."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=200,  # Suficientemente amplio para análisis completo pero limitado para respuestas cortas.
                temperature=0.7  # Mantener un tono algo creativo pero controlado.
            )
            return respuesta.choices[0].message.content.strip()
        except Exception as e:
            print(f"Error al generar insights: {e}")
            return "No se pudo generar análisis."


class MarketplaceScraper:
    def __init__(self, driver_path):
        self.driver_path = driver_path

    def configurar_driver(self):
        chrome_options = Options()
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        service = Service(self.driver_path)
        return webdriver.Chrome(service=service, options=chrome_options)

    def obtener_datos_Marketplace(self, query, location, limite_productos=100):
        """
        Obtiene productos de Marketplace con scroll dinámico y clic en "Ver más productos".

        Args:
            query (str): El término de búsqueda.
            location (tuple): Tupla con latitud y longitud (lat, lon).
            limite_productos (int): Límite máximo de productos a obtener.

        Returns:
            list: Lista de productos encontrados.
        """
        query_encoded = urllib.parse.quote(query)
        url = (
            f"https://es.MARKETPLACE-DEMO.com/app/search?"
            f"filters_source=search_box&keywords={query_encoded}&latitude={location[0]}&longitude={location[1]}"
        )
        print(f"Accediendo a: {url}")

        driver = self.configurar_driver()
        productos = []
        total_elementos = 0

        try:
            driver.get(url)
            # Limitamos la búsqueda para cumplir con buenas prácticas y respetar la plataforma
            while total_elementos < limite_productos:
                # Esperar a que los productos estén cargados
                WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, "ItemCardList__item"))
                )
                elementos = driver.find_elements(By.CLASS_NAME, "ItemCardList__item")
                print(f"Se encontraron {len(elementos)} elementos en la página.")

                # Procesar elementos nuevos
                for item in elementos[total_elementos:]:
                    try:
                        titulo = item.get_attribute("title") or "Sin título"
                        precio = item.find_element(By.CLASS_NAME, "ItemCard__price--bold").text
                        link = item.get_attribute("href") or ""
                        productos.append({"titulo": titulo, "precio": precio, "link": link})
                    except Exception as e:
                        print(f"Error al procesar un elemento: {e}")

                total_elementos = len(productos)
                print(f"Total productos obtenidos hasta ahora: {total_elementos}")

                # Salir si se alcanza el límite de productos
                if total_elementos >= limite_productos:
                    break

                # Intentar hacer clic en el botón "Ver más productos"
                try:
                    boton_ver_mas = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.ID, "btn-load-more"))
                    )
                    driver.execute_script("arguments[0].click();", boton_ver_mas)
                    time.sleep(2)  # Esperar que se carguen más productos
                except Exception as e:
                    print("No se pudo hacer clic en 'Ver más productos' o no hay más productos disponibles.")
                    break

        except Exception as e:
            print(f"Error general al obtener datos: {e}")
        finally:
            driver.quit()

        return productos

class TextUtils:
    @staticmethod
    def limitar_ascii(texto):
        texto_normalizado = unicodedata.normalize('NFD', texto)
        texto_sin_tildes = ''.join(
            c for c in texto_normalizado if unicodedata.category(c) != 'Mn'
        )
        return re.sub(r'[^\x00-\x7F]+', '', texto_sin_tildes)

if __name__ == "__main__":
    # Configura tu clave de OpenAI y la ruta de ChromeDriver
    API_KEY = "Tu APIkey"
    DRIVER_PATH = "C:/chromedriver-win64/chromedriver-win64/chromedriver.exe"

    query = input("Introduce el producto que deseas buscar: ")
    latitud = input("Introduce la latitud de tu ubicación: ")
    longitud = input("Introduce la longitud de tu ubicación: ")

    openai_handler = OpenAIHandler(api_key=API_KEY)
    scraper = MarketplaceScraper(driver_path=DRIVER_PATH)

    print("Buscando productos en Marketplace...")
    productos = scraper.obtener_datos_Marketplace(query, (latitud, longitud), limite_productos=100)


    if productos:
        print("\nProductos obtenidos de Marketplace:")
        for producto in productos:
            print(f"{producto['titulo']} - {producto['precio']} - {producto['link']}")

        print(f"\nSe encontraron {len(productos)} productos. Filtrando por relevancia...")
        productos_filtrados = openai_handler.filtrar_productos_por_tema(productos, query)

        if productos_filtrados:
            print(f"Se encontraron {len(productos_filtrados)} productos relevantes.")
            for producto in productos_filtrados:
                print(f"{producto['titulo']} - {producto['precio']} - {producto['link']}")

            print("\nGenerando análisis e insights...")
            insights = openai_handler.generar_insights(productos_filtrados, query)
            print("Análisis e insights:")
            print(insights)
        else:
            print("No se encontraron productos relevantes.")
    else:
        print("No se encontraron productos en Marketplace.")
